In [30]:
from datetime import datetime
from datetime import timedelta
import requests
import os
import json
import Bearer
import pytz
import time


In [31]:
def dt2localiso(dt):
    if dt is None:
        return ''
    
    dt = pytz.utc.localize(dt).astimezone(pytz.timezone("Asia/Tokyo"))
    return dt.isoformat()

In [32]:
# 開始日
start_time = datetime(2021, 3, 26, 19, 0, 0) # バイオハザード

# 終了日
end_time = start_time + timedelta(days=1)

start_time = dt2localiso(start_time)
end_time = dt2localiso(end_time)

In [33]:
def dt2localiso(dt):
    if dt is None:
        return ''
    
    dt = pytz.utc.localize(dt).astimezone(pytz.timezone("Asia/Tokyo"))
    return dt.isoformat()

In [34]:
# To set your environment variables in your terminal run the following line:
# export 'BEARER_TOKEN'='<your_bearer_token>'
bearer_token = Bearer.BEARER

search_url = "https://api.twitter.com/2/tweets/search/all"



def create_headers(bearer_token):
    headers = {"Authorization": "Bearer {}".format(bearer_token)}
    return headers

def connect_to_endpoint(url, headers, params):
    response = requests.request("GET", search_url, headers=headers, params=params)
    print(response.status_code)
    if response.status_code != 200:
        raise Exception(response.status_code, response.text)
    return response.json()



In [35]:
# Optional params: start_time,end_time,since_id,until_id,max_results,next_token,
# expansions,tweet.fields,media.fields,poll.fields,place.fields,user.fields
query_params = {'query': '#金曜ロードSHOW',
                'tweet.fields': "created_at,author_id", # スペースをいれない
                'max_results' : 100,
                "start_time":start_time ,
               "end_time": end_time}

json_response = None
query_params.pop("until_id", None)

for i in range(5):
    time.sleep(1) # 連続するとエラーになる
    if json_response:
        print(i)
        query_params.pop("start_time", None)
        query_params.pop("end_time", None)
        query_params["until_id"] = int(json_response["data"][-1]["id"])
        
        print(query_params["until_id"])
#         query_params["untie_id"] = int(json_response["data"][-1]["id"])+10000
    
    
    json_response = connect_to_endpoint(search_url, 
                                    create_headers(bearer_token), # headerを作る
                                    query_params,
                                    )
    for i,tmp_content in enumerate(json_response["data"]):
        if i % 10 == 0:
            created = iso_to_dt(tmp_content["created_at"])
#             print(i, tmp_content["text"], tmp_content["id"])
            print(tmp_content["id"], created)
    

200
1375863993420574727 2021-03-28 02:35:13+09:00
1375776927953211399 2021-03-27 20:49:14+09:00
1375746014938718213 2021-03-27 18:46:24+09:00
1375715369810231298 2021-03-27 16:44:38+09:00
1375697064043061251 2021-03-27 15:31:53+09:00
1375679652362997760 2021-03-27 14:22:42+09:00
1375671441543340039 2021-03-27 13:50:05+09:00
1375634750124085252 2021-03-27 11:24:17+09:00
1375595629775253507 2021-03-27 08:48:50+09:00
1375577044927488005 2021-03-27 07:34:59+09:00
1
1375563210519060482
200
1375563198573637632 2021-03-27 06:39:57+09:00
1375546715428691968 2021-03-27 05:34:28+09:00
1375516072258981888 2021-03-27 03:32:42+09:00
1375492692579246081 2021-03-27 01:59:47+09:00
1375486282529337345 2021-03-27 01:34:19+09:00
1375479495340089362 2021-03-27 01:07:21+09:00
1375474995866955776 2021-03-27 00:49:28+09:00
1375470959088476164 2021-03-27 00:33:26+09:00
1375467987725996042 2021-03-27 00:21:37+09:00
1375465815374323712 2021-03-27 00:12:59+09:00
2
1375463870668529664
200
1375463735985201160 2021

In [36]:
def iso_to_dt(iso_str):
    dt = None
    try:
        dt = datetime.strptime(iso_str, '%Y-%m-%dT%H:%M:%S.%fZ')
        dt = pytz.utc.localize(dt).astimezone(pytz.timezone("Asia/Tokyo"))
    except ValueError:
        try:
            dt = datetime.strptime(iso_str, '%Y-%m-%dT%H:%M:%S.%f%z')
            dt = dt.astimezone(pytz.timezone("Asia/Tokyo"))
        except ValueError:
            pass
    return dt